## Dependencies

In [1]:
# First thing to do was install the UC Irvine Repository and keras tuner module for later use in the CoLab notebook. 
!pip install ucimlrepo
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.9 MB/s eta 0:00:00


In [2]:
# We will be using Scikit learn and Tensorflow, so we will import those here.
import sklearn as skl
import tensorflow as tf

In [3]:
# Now to collect the dataset from the UC Irvine Repository.
from ucimlrepo import fetch_ucirepo

# fetch dataset
poker_hand = fetch_ucirepo(id=158)

# data (as pandas dataframes)
X = poker_hand.data.features
y = poker_hand.data.targets


# variable information
print(poker_hand.variables)






     name     role     type demographic description units missing_values
0      S1  Feature  Integer        None        None  None             no
1      C1  Feature  Integer        None        None  None             no
2      S2  Feature  Integer        None        None  None             no
3      C2  Feature  Integer        None        None  None             no
4      S3  Feature  Integer        None        None  None             no
5      C3  Feature  Integer        None        None  None             no
6      S4  Feature  Integer        None        None  None             no
7      C4  Feature  Integer        None        None  None             no
8      S5  Feature  Integer        None        None  None             no
9      C5  Feature  Integer        None        None  None             no
10  CLASS   Target  Integer        None        None  None             no


In [4]:
# metadata
print(poker_hand.metadata)


{'uci_id': 158, 'name': 'Poker Hand', 'repository_url': 'https://archive.ics.uci.edu/dataset/158/poker+hand', 'data_url': 'https://archive.ics.uci.edu/static/public/158/data.csv', 'abstract': 'Purpose is to predict poker hands', 'area': 'Games', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1025010, 'num_features': 10, 'feature_types': ['Categorical', 'Integer'], 'demographics': [], 'target_col': ['CLASS'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2002, 'last_updated': 'Sat Mar 09 2024', 'dataset_doi': '10.24432/C5KW38', 'creators': ['Robert Cattral', 'Franz Oppacher'], 'intro_paper': None, 'additional_info': {'summary': 'Each record is an example of a hand consisting of five playing cards drawn from a standard deck of 52. Each card is described using two attributes (suit and rank), for a total of 10 predictive attributes. There is one Class attribute that describes the "Poker Hand". T

In [5]:
# This will prepare the data for testing and training of the nerual network.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=9)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [6]:
# Because our model will need to account for 10 classification targets, we will need to use this utility to
# categorize the different y targets properly within the dataset, so that all 10 targets are possible for
# the model to train and predict on.
from tensorflow.keras.utils import to_categorical
number_classes = 10
y_train = to_categorical(y_train, number_classes)
y_test = to_categorical(y_test, number_classes)

In [7]:
# A quick test to ensure the categorization worked.
y_train[1]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
# We would like to use the tuner to help us determine the best DNN model to create. So, we
# take the following steps to get the tuner operational:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
  nn_model = tf.keras.models.Sequential()
    # Allow kerastuner to decide which activation function to use in hidden layers
  activation = hp.Choice('activation', ['relu', 'tanh'])
    # Allow kerastuner to decide number of neurons in first layer
  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
      min_value=1 , max_value=50, step=10), activation=activation, input_dim=10))
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
  for i in range(hp.Int('num_layers', 1, 5)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
    min_value=1,
    max_value=50,
    step=10),
    activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    # Compile the model
  nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  return nn_model

In [9]:
# Import the kerastuner library
import keras_tuner as kt
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=10,
    hyperband_iterations=1
)

In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled, y_test))

Trial 30 Complete [00h 09m 04s]
val_accuracy: 0.5233577489852905

Best val_accuracy So Far: 0.9980565905570984
Total elapsed time: 02h 08m 43s


In [11]:
# Get top 3 model hyperparameters and print the values
best_hyper = tuner.get_best_hyperparameters(3)
for param in best_hyper:
  print(param.values)

{'activation': 'relu', 'first_units': 21, 'num_layers': 2, 'units_0': 41, 'units_1': 21, 'units_2': 11, 'units_3': 1, 'units_4': 21, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0013'}
{'activation': 'relu', 'first_units': 11, 'num_layers': 5, 'units_0': 41, 'units_1': 21, 'units_2': 1, 'units_3': 31, 'units_4': 21, 'tuner/epochs': 10, 'tuner/initial_epoch': 4, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
{'activation': 'relu', 'first_units': 21, 'num_layers': 2, 'units_0': 41, 'units_1': 21, 'units_2': 11, 'units_3': 1, 'units_4': 21, 'tuner/epochs': 4, 'tuner/initial_epoch': 2, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0003'}


In [12]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
  model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
  print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8008/8008 - 10s - loss: 0.0132 - accuracy: 0.9981 - 10s/epoch - 1ms/step
Loss: 0.013169246725738049, Accuracy: 0.9980565905570984
8008/8008 - 10s - loss: 0.0261 - accuracy: 0.9937 - 10s/epoch - 1ms/step
Loss: 0.026071233674883842, Accuracy: 0.9937132596969604
8008/8008 - 9s - loss: 0.0686 - accuracy: 0.9778 - 9s/epoch - 1ms/step
Loss: 0.06858868896961212, Accuracy: 0.9777914881706238
